In [117]:
# -*- coding: utf-8 -*-
import json

def j2p(f, c=[0]):
    '''Converts jupyter to python
    Args:
        (f) = jupyter ipynb filename as string WITHOUT extension e.g. 'helper'
        (c) = list of cells e.g. [0, 1]'''

    with open(f+'.ipynb') as data_file:
        code = json.load(data_file)
    
    code_cells = [cell['source'] for cell in code['cells'] if cell['cell_type'] == 'code']
    cells = [code_cells[i] for i in c]
    with open(f+'.py', 'w') as data:
        # write line not having util.startLoop()
        [data.write("%s\n" % (line)) for lines in cells for line in lines if not 'util.startLoop()' in line]

In [120]:
j2p('helper', [1])